<a href="https://colab.research.google.com/github/Veronika2311/ML_CL_HSE/blob/main/HW_class2_Regression_Texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На этом занятии мы попробуем задачу регрессии. Данные возьмем вот эти - https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Перед обучением обучением модели, нужно подготовить данные:

- найти\собрать данные
- почистить и предобработать
- преобразовать в матрицы


In [1]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [3]:
import gdown
!gdown 1r2lQgJiFQUTQ5sRC2H5SQx39VE1HoMyZ

Downloading...
From: https://drive.google.com/uc?id=1r2lQgJiFQUTQ5sRC2H5SQx39VE1HoMyZ
To: /content/IMDB Dataset.csv
100% 66.2M/66.2M [00:00<00:00, 126MB/s]


In [6]:
data = pd.read_csv('IMDB Dataset.csv')
print(data.shape)

data.head(3)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


## Что делать с NaN?
Есть 3 варианта

In [7]:
# 1. Убрать строки с NaN
print(data.isna().any())
data.shape

review       False
sentiment    False
dtype: bool


(50000, 2)

In [8]:
print(data.shape)
tmp = data.dropna()
tmp.shape

(50000, 2)


(50000, 2)

In [9]:
# 2. Превратить NaN в 0
print(data.shape)
tmp = data.fillna(0)
print(tmp.shape)

(50000, 2)
(50000, 2)


In [10]:
# 3. Превратить NaN в средние значения по колонке

# вычисляем средние для колонок с пустыми значениями
meta_mean = data.Metascore.mean()
rev_mean = data['Revenue (Millions)'].mean()

#заменяем пустоты на средние значения
data.Metascore.fillna(meta_mean, inplace=True)
data['Revenue (Millions)'].fillna(rev_mean, inplace=True)

# проверяем присутствие NaN
data.isna().any()

AttributeError: ignored

## Подготовка данных

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов

Колонка "Rating" станет **целевой переменной, или таргетом** (y)<br>
Остальных данные будут **обучающей выборкой** (X)

In [ ]:
data.Description

In [ ]:
# подготовим описания фильмов
data["text"] = data.Description.apply(lambda x: x.lower().split())

data["text"]

In [ ]:
data.text.values

In [ ]:
input_text = list(data.text.values)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[10:12]

обучаем модель на текстах описаний фильмов (можно поизменять параметры)

In [ ]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [ ]:
model.save("D2V.model") # сохранение модели

In [ ]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model.dv[documents[0].tags[0]]


Теперь нужно добавить векторы в датасет с остальными параметрами

In [ ]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model.dv[x.tags][0])
    vectors.append(vec)

In [ ]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

split_df


In [ ]:
# теперь добавим его к основному датафрейму
result = data_imdb_1.join(split_df, how='left')
result.shape

In [ ]:
result

In [ ]:
# переопределим датасет, оставив только важное

data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5"]
              ]


data_sm.head(3)

## Подготавливаем матрицы

In [ ]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values

display(X, X.shape)

In [ ]:
data_sm.isna().any()

In [ ]:
y = data_sm['Rating'].values # отдельно вынесли массив со значениями скорости ветра
y.shape

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения.
Для этого используется StandardScaler.

До нормализации:

In [ ]:
list(X[0])

In [ ]:
# использзуем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

После:

In [ ]:
list(sc.fit_transform(X)[0])

теперь с данными удобнее работать и обучать

In [ ]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor = Ridge()


# обучаем
regressor.fit(X_train, y_train)

In [ ]:
# давайте предскажем результат для тестовой выборки

y_preds = regressor.predict(X_test)

### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

In [ ]:
mean_absolute_error(y_test, y_preds)

Попробуйте разные значения для параметра регуляризации alpha при обучении модели. Как они влияют на величину ошибки?